In [6]:
import requests
import pandas as pd

In [7]:
df = pd.read_csv('notebooks/claims.csv')

In [8]:
import csv
import json

# Define the path to your CSV file
csv_file_path = 'notebooks/claims.csv'
json_file_path = 'notebooks/documents.json'

# Initialize a list to hold the JSON data
data_list = []

# Read the CSV file and convert each row to a dictionary
with open(csv_file_path, mode='r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        data_list.append(row)

# Write the list of dictionaries to a JSON file
with open(json_file_path, mode='w', encoding='utf-8') as json_file:
    json.dump(data_list, json_file, indent=4)

print(f'CSV has been converted to JSON and saved as {json_file_path}')


CSV has been converted to JSON and saved as notebooks/documents.json


In [5]:
# docs_url = 'https://raw.githubusercontent.com/KayDD1/uk-benefits-assistant/main/notebooks/documents.json'
# docs_response = requests.get(docs_url)
# documents_raw = docs_response.json()

# documents = []

import requests
import json

# Load the JSON data
with open('your_output_json_file.json', 'r', encoding='utf-8') as json_file:
    json_data = json.load(json_file)

# Example POST request
response = requests.post('https://your-api-endpoint.com', json=json_data)

# Print response from the server
print(response.status_code)
print(response.json())


JSONDecodeError: Extra data: line 2 column 1 (char 207)

In [ ]:
df.columns = df.columns.str.lower()
df

In [ ]:
documents = df.to_dict(orient='records')

In [ ]:
documents[0]

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [ ]:
len(model.encode("Getting size of model dim"))

In [ ]:
sections = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["category"]).tolist()
    sections.append(doc)

In [ ]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "answer": {"type": "text"},
            "question": {"type": "text"},
            "category": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 384, "index": True, "similarity": "cosine"},
        }
    }
}
index_name = "uk-benefit-claims"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
for doc in sections:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [ ]:
search_term = "How do I stop claiming benefits?"
vector_search_term = model.encode(search_term)

In [ ]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [ ]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

In [ ]:
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}
response = es_client.search(
    index=index_name,
    query={
        "match": {"category": "Manage existing benefit"},
    },
    knn=knn_query,
    size=5
)
response["hits"]["hits"]